In [70]:
! pip install -qU -r requirements.txt

In [48]:
from decouple import AutoConfig
from icecream import ic
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, trim_messages
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [49]:
config = AutoConfig(search_path='../notes-rag/')

GROQ_API_KEY = config("GROQ_API_KEY")
LANGCHAIN_TRACING_V2 = config("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = config("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT = config("LANGCHAIN_PROJECT")
lANGCHAIN_API_KEY = config("LANGCHAIN_API_KEY")

In [50]:
model = ChatGroq(model="gemma2-9b-it", api_key=GROQ_API_KEY)
output_parser = StrOutputParser()

In [51]:
response = model.invoke("Hi, My name is Mano, I am a chief Data Scientist")
print(output_parser.invoke(response))

Hello Mano, it's nice to meet you!

Being a Chief Data Scientist is a fascinating role. What kind of projects are you currently working on? 

I'm eager to learn more about your work and how I can be of assistance. Perhaps you have some data science questions or tasks you'd like to discuss?



In [52]:
message = [
    HumanMessage(content="Hi, My name is Mano, I am a chief Data Scientist"),
    AIMessage(content="Hi Mano, it's nice to meet you! \n\nThat's impressive, Chief Data Scientist is a great role. What kind of projects are you currently working on?  I'm always interested in learning about the latest developments in data science.\n"),
    HumanMessage(content="Hey, what was my name ?")
]

In [53]:
output_parser.invoke(model.invoke(message))

'Your name is Mano.  😊  \n\nI remember that from our introduction!  How can I help you today?  \n'

In [54]:
store = dict()

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [55]:
config={
    "configurable": {
        "session_id": "chat_1"
    }
}

response = with_message_history.invoke([
    HumanMessage(content="Hi, My name is Mano, I am a chief Data Scientist")
], config=config)
print(response, end="\n\n")

response = with_message_history.invoke([
    HumanMessage(content="what is my name?")
], config=config)
print(response)

content="Hi Mano, it's nice to meet you! \n\nAs a large language model, I'm always interested in learning about people working in exciting fields like data science. What kind of projects are you working on these days?  \n\nDo you have any specific questions for me about data science or AI that I could help you with?\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 22, 'total_tokens': 94, 'completion_time': 0.130909091, 'prompt_time': 0.000171169, 'queue_time': 0.01434392, 'total_time': 0.13108026}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-c1cd5e27-2af1-4b7c-9cec-71877e30f05a-0' usage_metadata={'input_tokens': 22, 'output_tokens': 72, 'total_tokens': 94}

content='Your name is Mano.  You told me at the beginning of our conversation! 😊 \n\n\nIs there anything else I can help you with, Mano?\n' additional_kwargs={} response_metadata={'token_usage': {'completio

In [56]:
config={
    "configurable": {
        "session_id": "chat_2"
    }
}

response = with_message_history.invoke([
    HumanMessage(content="what is my name?")
], config=config)
print(response)

content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 14, 'total_tokens': 52, 'completion_time': 0.069090909, 'prompt_time': 8.888e-05, 'queue_time': 0.014206630000000001, 'total_time': 0.069179789}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-5cb8bb66-377b-44e1-944a-2585e7c13e9a-0' usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52}


In [57]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant, answer all of the questions to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model

In [58]:
chain.invoke({"messages": [
    HumanMessage(content="Hi my name is Hua Mulan")
]})

AIMessage(content="Hello Hua Mulan! It's nice to meet you. \n\nI'm happy to answer any questions you have to the best of my ability.  What can I help you with today?  😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 32, 'total_tokens': 79, 'completion_time': 0.085454545, 'prompt_time': 0.000366618, 'queue_time': 0.014005032, 'total_time': 0.085821163}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1f73266f-3c59-4d04-9841-d3b69a5d4d34-0', usage_metadata={'input_tokens': 32, 'output_tokens': 47, 'total_tokens': 79})

In [59]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [60]:
config={
    "configurable": {
        "session_id": "chat_3"
    }
}

response = with_message_history.invoke({"messages": [
    HumanMessage(content="Hi my name is Hua Mulan")
]}, config)
print(response, end="\n\n")

response = with_message_history.invoke({"messages": [
    HumanMessage(content="whats my name, AI ?")
]}, config)
print(response, end="\n\n")

response = with_message_history.invoke({"messages": [
    HumanMessage(content="based on my name, guess my gender ?")
]}, config)
print(response)

content="Hello Hua Mulan!  It's nice to meet you.  \n\nWhat can I do for you today? I'm ready for any questions you have, big or small.  😊  \n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 32, 'total_tokens': 76, 'completion_time': 0.08, 'prompt_time': 0.000310599, 'queue_time': 0.013343519, 'total_time': 0.080310599}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-034063c4-4c95-470b-b32a-8e965559d1f0-0' usage_metadata={'input_tokens': 32, 'output_tokens': 44, 'total_tokens': 76}

content='Your name is Hua Mulan.  You told me yourself when you first introduced yourself!  😄 \n\n\nIs there anything else I can help you with, Mulan?\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 89, 'total_tokens': 125, 'completion_time': 0.065454545, 'prompt_time': 0.0022815, 'queue_time': 0.012104509999999999, 'total_time': 0

In [61]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant, answer all of the questions to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model

In [62]:
response = chain.invoke({
    "messages": [HumanMessage(content="Hi my name is Arnold")],
    "language": "telugu"
})

print(response)

content='మీకు నమస్కరాలు, ఆర్నోల్డ్! 😊 \n\nనేను మీకు సహాయకరిగా వ్యవహరిస్తాను. మీరు ఏవైనా ప్రశ్నలు అడిగితే నేను నా శక్తి ఉన్నంత విధంగా జవాబు ఇస్తాను.  \n\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 34, 'total_tokens': 116, 'completion_time': 0.149090909, 'prompt_time': 0.000329029, 'queue_time': 0.014961469, 'total_time': 0.149419938}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-d7a9c8f2-ec9f-4583-874e-d331c2cee746-0' usage_metadata={'input_tokens': 34, 'output_tokens': 82, 'total_tokens': 116}


In [63]:
with_message_history = RunnableWithMessageHistory(chain,  get_session_history, input_messages_key="messages")

In [64]:
config={
    "configurable": {
        "session_id": "chat_4"
    }
}

response = with_message_history.invoke({
    "messages": [HumanMessage(content="Hi my name is Shankar mahadevan")],
    "language": "telugu"
}, config)
print(response, end="\n\n")

response = with_message_history.invoke({
    "messages": [HumanMessage(content="what is my name ?")],
    "language": "telugu"
}, config)
print(response, end="\n\n")

response = with_message_history.invoke({
    "messages": [HumanMessage(content="whats my gender ?")],
    "language": "telugu"
}, config)
print(response, end="\n\n")

content='నేను మీకు సహాయకరిగా ఉంటాను, మీ ప్రశ్నలన్నీ నేను తెలుగులో సరిగ్గా జవాబు ఇస్తాను. \n\nశంకర్ మహదేవన్ అని మీ పేరు అనేది నాకు తెలిసింది.  \n\nఏ ప్రశ్నలు అడిగితే, నేను సమాధానం చెబుతాను. 😊 \n\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 37, 'total_tokens': 143, 'completion_time': 0.192727273, 'prompt_time': 0.000309828, 'queue_time': 0.014083942, 'total_time': 0.193037101}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-bc1847a7-3b4b-48ff-b8b6-c5e28da58e9d-0' usage_metadata={'input_tokens': 37, 'output_tokens': 106, 'total_tokens': 143}

content='మీ పేరు శంకర్ మహదేవన్.  \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 155, 'total_tokens': 173, 'completion_time': 0.032727273, 'prompt_time': 0.004882388, 'queue_time': 0.00949018, 'total_time': 0.037609661}, 'model_name': 'gemma2-9b-it', 'system_fingerprint

In [65]:
trimmer = trim_messages(max_tokens=100, strategy="last", token_counter=model, include_system=True, allow_partial=False, start_on="human")

In [66]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi, i am bob"),
    AIMessage(content="Hi bob"),
    HumanMessage(content="i like vanilla icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun ?"),
    AIMessage(content="yes")
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi, i am bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi bob', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i like vanilla icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={})]

In [67]:
chain = (
    RunnablePassthrough.assign(messages= itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke({
    "messages": messages + [HumanMessage(content="what ice cream do i like ?")],
    "language": "telugu"
})

print(response)

content='మీరు వేనిల ICE CREAM ఇష్టపడతారు. 🍦  \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 119, 'total_tokens': 141, 'completion_time': 0.04, 'prompt_time': 0.003539974, 'queue_time': 0.011073104, 'total_time': 0.043539974}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-bf4b2f77-ec00-4360-b284-de97e244415d-0' usage_metadata={'input_tokens': 119, 'output_tokens': 22, 'total_tokens': 141}


In [68]:
with_message_history = RunnableWithMessageHistory(chain,  get_session_history, input_messages_key="messages")

In [69]:
config={
    "configurable": {
        "session_id": "chat_5"
    }
}

response = with_message_history.invoke({
    "messages": [HumanMessage(content="what math problem did i ask for ?")],
    "language": "telugu"
}, config)
print(response, end="\n\n")

response = with_message_history.invoke({
    "messages": [HumanMessage(content="do i like ice cream ?")],
    "language": "telugu"
}, config)
print(response, end="\n\n")

response = with_message_history.invoke({
    "messages": [HumanMessage(content="whats my gender ?")],
    "language": "telugu"
}, config)
print(response, end="\n\n")

KeyboardInterrupt: 